This example is from Google Qwiklabs' "Ingesting FHIR Data with the HealthcareAPI," which can be found at this link:
https://www.qwiklabs.com/focuses/6104?parent=catalog

### Define Variables

```
export PROJECT_ID=$(gcloud config list --format 'value(core.project)')
export PROJECT_NUMBER=$(gcloud projects list --filter="${PROJECT_ID}" --format="value(PROJECT_NUMBER)")
export REGION=us-central1
export DATASET_ID=dataset1
export FHIR_STORE_ID=fhirstore1
export TOPIC=fhir-topic
```

### Navigate to Healthcare API

* Create dataset: 
    * Name = "dataset1"
    * Region = "us-central1
* Select dataset1
* Create Data Store
    * Select FHIR
    * Name = "fhirstore1"
    * Create Topic
        * Topic ID = "fhir-topic"
        * Click Create Topic
    * Select FHIR Store Configuration = "R4"
    * Click Create
* Create another Data Store
    * Select FHIR
    * Name = "de_id
    * Select FHIR Store Configuration = "R4"
    * Click Create

### Create BigQuery datasets

Command 1: Create dataset1
```
bq --location=us-east1 mk --dataset --description HCAPI-dataset $PROJECT_ID:$DATASET_ID
```
Command 2: Create de_id
```
bq --location=us-east1 mk --dataset --description HCAPI-dataset-de-id $PROJECT_ID:de_id
```

### Allow healthcare API to export to BigQuery

Command 1
```
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member=serviceAccount:service-$PROJECT_NUMBER@gcp-sa-healthcare.iam.gserviceaccount.com \
--role=roles/bigquery.dataEditor
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member=serviceAccount:service-$PROJECT_NUMBER@gcp-sa-healthcare.iam.gserviceaccount.com \
--role=roles/bigquery.jobUser
```

Command 2
```
curl -X PATCH \
    -H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    --data "{
      'streamConfigs': [
        {
          'bigqueryDestination': {
            'datasetUri': 'bq://$PROJECT_ID.$DATASET_ID',
            'schemaConfig': {
              'schemaType': 'ANALYTICS'
            }
          }
        }
      ]
    }" "https://healthcare.googleapis.com/v1/projects/$PROJECT_ID/locations/$REGION/datasets/$DATASET_ID/fhirStores/$FHIR_STORE_ID?updateMask=streamConfigs"
```

### Import FHIR data into healthcare data

```
gcloud healthcare fhir-stores import gcs $FHIR_STORE_ID --dataset=$DATASET_ID --location=$REGION --gcs-uri=gs://spls/gsp457/fhir_devdays_gcp/fhir1/* --content-structure=BUNDLE_PRETTY
```

### De-identifying the data

* Navigate to Healthcare API
* Select dataset 1
* Select "Actions" dropdown for fhirstore1
    * Select de-identify
    * Select dataset 1 as the dataset
    * Select de_id as the destination
    * Click to append the data
    * Click next
    * Click de-identify

### Export to BigQuery

Command 1: export dataset1
```
gcloud beta healthcare fhir-stores export bq $FHIR_STORE_ID --dataset=$DATASET_ID --location=$REGION --bq-dataset=bq://$PROJECT_ID.$DATASET_ID --schema-type=analytics
```

Command 2: export de_id
```
gcloud beta healthcare fhir-stores export bq de_id --dataset=$DATASET_ID --location=$REGION --bq-dataset=bq://$PROJECT_ID.de_id --schema-type=analytics
```

### Example queries

Query 1: View patient data with name
```sql
SELECT   
id AS patient_id,
  name[safe_offset(0)].given AS given_name,
  name[safe_offset(0)].family AS family,
  birthDate AS birth_date
  FROM `dataset1.Patient` LIMIT 10
```


Query 2: View patient name with removed (de-identified) name
```sql
SELECT   
id AS patient_id,
  name[safe_offset(0)].given AS given_name,
  name[safe_offset(0)].family AS family,
  birthDate AS birth_date
  FROM `de_id.Patient` LIMIT 10
```